In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
url = 'https://pokeapi.co/api/v2/pokemon/'
#create a list of numbers from 1 to 1025
numbers = list(range(1, 1026))

In [4]:
#make a GET request to fetch the raw JSON content from the URL + each number in the list
response = [requests.get(url + str(number)) for number in numbers]
pokemon_json = [response[i].json() for i in range(len(response))]


In [5]:
pokemon = [pokemon_json[i]['name'] for i in range(len(pokemon_json))]
# put the pokemon names in a pandas dataframe
pokemon_df = pd.DataFrame(pokemon, columns=['pokemon'])
pokemon_df


,pokemon
0,bulbasaur
1,ivysaur
2,venusaur
3,charmander
4,charmeleon
...,...
1020,raging-bolt
1021,iron-boulder
1022,iron-crown
1023,terapagos


In [6]:
#pull out the abilities from the JSON
abilities = [pokemon_json[i]['abilities'] for i in range(len(pokemon_json))]
abilities = [[abilities[i][j]['ability']['name'] for j in range(len(abilities[i]))] for i in range(len(abilities))]
abilities
#put the abilities in the pokemon dataframe make it ability one, ability two, etc
abilities_df = pd.DataFrame(abilities)
abilities_df.columns = ['ability_one', 'ability_two', 'ability_three']
pokemon_df = pd.concat([pokemon_df, abilities_df], axis
=1)
pokemon_df

,pokemon,ability_one,ability_two,ability_three
0,bulbasaur,overgrow,chlorophyll,None
1,ivysaur,overgrow,chlorophyll,None
2,venusaur,overgrow,chlorophyll,None
3,charmander,blaze,solar-power,None
4,charmeleon,blaze,solar-power,None
...,...,...,...,...
1020,raging-bolt,protosynthesis,None,None
1021,iron-boulder,quark-drive,None,None
1022,iron-crown,quark-drive,None,None
1023,terapagos,tera-shift,None,None


In [7]:
#find the height
height = [pokemon_json[i]['height'] for i in range(len(pokemon_json))]
#put the height in the pokemon dataframe
pokemon_df['height'] = height
pokemon_df

,pokemon,ability_one,ability_two,ability_three,height
0,bulbasaur,overgrow,chlorophyll,None,7
1,ivysaur,overgrow,chlorophyll,None,10
2,venusaur,overgrow,chlorophyll,None,20
3,charmander,blaze,solar-power,None,6
4,charmeleon,blaze,solar-power,None,11
...,...,...,...,...,...
1020,raging-bolt,protosynthesis,None,None,52
1021,iron-boulder,quark-drive,None,None,15
1022,iron-crown,quark-drive,None,None,16
1023,terapagos,tera-shift,None,None,2


In [8]:
#find the weight
weight = [pokemon_json[i]['weight'] for i in range(len(pokemon_json))]
#put the weight in the pokemon dataframe
pokemon_df['weight'] = weight
pokemon_df

,pokemon,ability_one,ability_two,ability_three,height,weight
0,bulbasaur,overgrow,chlorophyll,None,7,69
1,ivysaur,overgrow,chlorophyll,None,10,130
2,venusaur,overgrow,chlorophyll,None,20,1000
3,charmander,blaze,solar-power,None,6,85
4,charmeleon,blaze,solar-power,None,11,190
...,...,...,...,...,...,...
1020,raging-bolt,protosynthesis,None,None,52,4800
1021,iron-boulder,quark-drive,None,None,15,1625
1022,iron-crown,quark-drive,None,None,16,1560
1023,terapagos,tera-shift,None,None,2,65


In [9]:
# find the moves
moves = [pokemon_json[i]['moves'] for i in range(len(pokemon_json))]
#pull out the moves from the JSON
moves = [[moves[i][j]['move']['name'] for j in range(len(moves[i]))] for i in range(len(moves))]
#put the moves in the pokemon dataframe
pokemon_df['moves'] = moves
#get the number of moves for each pokemon
num_moves = [len(moves[i]) for i in range(len(moves))]
#put the number of moves in the pokemon dataframe
pokemon_df['num_moves'] = num_moves
pokemon_df

,pokemon,ability_one,ability_two,ability_three,height,weight,moves,num_moves
0,bulbasaur,overgrow,chlorophyll,None,7,69,"[razor-wind, swords-dance, cut, bind, vine-whi...",86
1,ivysaur,overgrow,chlorophyll,None,10,130,"[swords-dance, cut, bind, vine-whip, headbutt,...",83
2,venusaur,overgrow,chlorophyll,None,20,1000,"[swords-dance, cut, bind, vine-whip, headbutt,...",96
3,charmander,blaze,solar-power,None,6,85,"[mega-punch, fire-punch, thunder-punch, scratc...",105
4,charmeleon,blaze,solar-power,None,11,190,"[mega-punch, fire-punch, thunder-punch, scratc...",103
...,...,...,...,...,...,...,...,...
1020,raging-bolt,protosynthesis,None,None,52,4800,"[stomp, body-slam, take-down, double-edge, roa...",55
1021,iron-boulder,quark-drive,None,None,15,1625,"[swords-dance, horn-attack, body-slam, take-do...",49
1022,iron-crown,quark-drive,None,None,16,1560,"[swords-dance, body-slam, take-down, double-ed...",49
1023,terapagos,tera-shift,None,None,2,65,"[headbutt, body-slam, take-down, double-edge, ...",59


In [10]:
#pull out the HP stat from the JSON
hp = [pokemon_json[i]['stats'][0]['base_stat'] for i in range(len(pokemon_json))]
#pull out the attack stat from the JSON
attack = [pokemon_json[i]['stats'][1]['base_stat'] for i in range(len(pokemon_json))]
#pull out the defense stat from the JSON
defense = [pokemon_json[i]['stats'][2]['base_stat'] for i in range(len(pokemon_json))]
#pull out the special attack stat from the JSON
special_attack = [pokemon_json[i]['stats'][3]['base_stat'] for i in range(len(pokemon_json))]
#pull out the special defense stat from the JSON
special_defense = [pokemon_json[i]['stats'][4]['base_stat'] for i in range(len(pokemon_json))]
#pull out the speed stat from the JSON
speed = [pokemon_json[i]['stats'][5]['base_stat'] for i in range(len(pokemon_json))]
#put the stats in the pokemon dataframe
pokemon_df['hp'] = hp
pokemon_df['attack'] = attack
pokemon_df['defense'] = defense
pokemon_df['special_attack'] = special_attack
pokemon_df['special_defense'] = special_defense
pokemon_df['speed'] = speed
pokemon_df

,pokemon,ability_one,ability_two,ability_three,height,weight,moves,num_moves,hp,attack,defense,special_attack,special_defense,speed
0,bulbasaur,overgrow,chlorophyll,None,7,69,"[razor-wind, swords-dance, cut, bind, vine-whi...",86,45,49,49,65,65,45
1,ivysaur,overgrow,chlorophyll,None,10,130,"[swords-dance, cut, bind, vine-whip, headbutt,...",83,60,62,63,80,80,60
2,venusaur,overgrow,chlorophyll,None,20,1000,"[swords-dance, cut, bind, vine-whip, headbutt,...",96,80,82,83,100,100,80
3,charmander,blaze,solar-power,None,6,85,"[mega-punch, fire-punch, thunder-punch, scratc...",105,39,52,43,60,50,65
4,charmeleon,blaze,solar-power,None,11,190,"[mega-punch, fire-punch, thunder-punch, scratc...",103,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,raging-bolt,protosynthesis,None,None,52,4800,"[stomp, body-slam, take-down, double-edge, roa...",55,125,73,91,137,89,75
1021,iron-boulder,quark-drive,None,None,15,1625,"[swords-dance, horn-attack, body-slam, take-do...",49,90,120,80,68,108,124
1022,iron-crown,quark-drive,None,None,16,1560,"[swords-dance, body-slam, take-down, double-ed...",49,90,72,100,122,108,98
1023,terapagos,tera-shift,None,None,2,65,"[headbutt, body-slam, take-down, double-edge, ...",59,90,65,85,65,85,60


In [11]:
#pull out the type from the JSON
types = [pokemon_json[i]['types'] for i in range(len(pokemon_json))]
#split the types into two separate columns
type_one = [types[i][0]['type']['name'] for i in range(len(types))]
type_two = [types[i][1]['type']['name'] if len(types[i]) > 1 else None for i in range(len(types))]
#put the types in the pokemon dataframe
pokemon_df['type_one'] = type_one
pokemon_df['type_two'] = type_two
pokemon_df

,pokemon,ability_one,ability_two,ability_three,height,weight,moves,num_moves,hp,attack,defense,special_attack,special_defense,speed,type_one,type_two
0,bulbasaur,overgrow,chlorophyll,None,7,69,"[razor-wind, swords-dance, cut, bind, vine-whi...",86,45,49,49,65,65,45,grass,poison
1,ivysaur,overgrow,chlorophyll,None,10,130,"[swords-dance, cut, bind, vine-whip, headbutt,...",83,60,62,63,80,80,60,grass,poison
2,venusaur,overgrow,chlorophyll,None,20,1000,"[swords-dance, cut, bind, vine-whip, headbutt,...",96,80,82,83,100,100,80,grass,poison
3,charmander,blaze,solar-power,None,6,85,"[mega-punch, fire-punch, thunder-punch, scratc...",105,39,52,43,60,50,65,fire,None
4,charmeleon,blaze,solar-power,None,11,190,"[mega-punch, fire-punch, thunder-punch, scratc...",103,58,64,58,80,65,80,fire,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,raging-bolt,protosynthesis,None,None,52,4800,"[stomp, body-slam, take-down, double-edge, roa...",55,125,73,91,137,89,75,electric,dragon
1021,iron-boulder,quark-drive,None,None,15,1625,"[swords-dance, horn-attack, body-slam, take-do...",49,90,120,80,68,108,124,rock,psychic
1022,iron-crown,quark-drive,None,None,16,1560,"[swords-dance, body-slam, take-down, double-ed...",49,90,72,100,122,108,98,steel,psychic
1023,terapagos,tera-shift,None,None,2,65,"[headbutt, body-slam, take-down, double-edge, ...",59,90,65,85,65,85,60,normal,None


In [12]:
#save the dataframe to a csv file
pokemon_df.to_csv('pokemon.csv', index=False)